**Actividad - Autoencoder para detección de fraude**

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score
)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

if '__file__' in globals():
    ruta_base = os.path.dirname(os.path.abspath(__file__))
else:
    ruta_base = os.getcwd()

ruta_datos = os.path.join(ruta_base, "..", "data", "application_train.csv")
datos = pd.read_csv(ruta_datos)

filas, columnas = datos.shape
tasa_incumplimiento = datos["TARGET"].mean() * 100
casos_normales = (datos["TARGET"] == 0).sum()
casos_incumplidos = (datos["TARGET"] == 1).sum()

print(f"\nArchivo cargado correctamente: {filas} filas y {columnas} columnas")
print(f"Tasa de incumplimiento: {tasa_incumplimiento:.2f}%")
print(f"Casos normales: {casos_normales} | Casos incumplidos: {casos_incumplidos}")




Archivo cargado correctamente: 307511 filas y 122 columnas
Tasa de incumplimiento: 8.07%
Casos normales: 282686 | Casos incumplidos: 24825


In [9]:
# SELECCIÓN DE VARIABLES Y CREACIÓN DE ATRIBUTOS DERIVADOS
# Variables de tiempo (convertir días a años)
datos['EDAD'] = -datos['DAYS_BIRTH'] / 365
datos['AÑOS_EMPLEADO'] = -datos['DAYS_EMPLOYED'] / 365
datos['AÑOS_EMPLEADO'] = datos['AÑOS_EMPLEADO'].replace(1000.67, np.nan)

# Ratios financieros básicos
datos['RATIO_CREDITO_INGRESO'] = datos['AMT_CREDIT'] / datos['AMT_INCOME_TOTAL']
datos['RATIO_ANUALIDAD_INGRESO'] = datos['AMT_ANNUITY'] / datos['AMT_INCOME_TOTAL']
datos['INGRESO_PER_CAPITA'] = datos['AMT_INCOME_TOTAL'] / (datos['CNT_FAM_MEMBERS'] + 1)

# Detectar inconsistencias sospechosas
datos['INCONSISTENCIA_SCORE'] = 0
mask1 = (datos['EXT_SOURCE_2'] < 0.3) & (datos['AMT_INCOME_TOTAL'] > datos['AMT_INCOME_TOTAL'].quantile(0.75))
datos.loc[mask1, 'INCONSISTENCIA_SCORE'] = 1

datos['INCONSISTENCIA_FAMILIA'] = 0
mask2 = (datos['CNT_FAM_MEMBERS'] >= 4) & (datos['AMT_INCOME_TOTAL'] < datos['AMT_INCOME_TOTAL'].quantile(0.25))
datos.loc[mask2, 'INCONSISTENCIA_FAMILIA'] = 1

datos['TOTAL_INCONSISTENCIAS'] = datos['INCONSISTENCIA_SCORE'] + datos['INCONSISTENCIA_FAMILIA']

# Variables de círculo social
datos['DEFAULTS_SOCIALES'] = (datos['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0) + 
                               datos['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0))
datos['TIENE_CIRCULO_RIESGOSO'] = (datos['DEFAULTS_SOCIALES'] > 0).astype(int)

# Flags de riesgo
datos['SOBREENDEUDAMIENTO'] = (datos['RATIO_CREDITO_INGRESO'] > 8).astype(int)
datos['JOVEN_CREDITO_ALTO'] = ((datos['EDAD'] < 25) & (datos['AMT_CREDIT'] > 500000)).astype(int)

# Seleccionar variables finales
variables_seleccionadas = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
    'RATIO_CREDITO_INGRESO', 'RATIO_ANUALIDAD_INGRESO', 'INGRESO_PER_CAPITA',
    'EXT_SOURCE_2', 'EXT_SOURCE_3',
    'EDAD', 'AÑOS_EMPLEADO', 'CNT_CHILDREN', 'CNT_FAM_MEMBERS',
    'REGION_RATING_CLIENT', 'DAYS_REGISTRATION',
    'TOTAL_INCONSISTENCIAS', 'INCONSISTENCIA_SCORE', 'INCONSISTENCIA_FAMILIA',
    'TIENE_CIRCULO_RIESGOSO', 'SOBREENDEUDAMIENTO', 'JOVEN_CREDITO_ALTO'
]

print(f"Total de variables creadas: {len(variables_seleccionadas)}")
print(f"Inconsistencias detectadas: {datos['TOTAL_INCONSISTENCIAS'].sum()}")

Total de variables creadas: 20
Inconsistencias detectadas: 14180
